#### In this notebook we will use mlp to predict next word 

In [30]:
import torch as t 
import torch.nn.functional as f
import matplotlib.pyplot as plt
%matplotlib inline
t.seed()

11888874264141112262

In [31]:
words = open('names.txt', 'r').read().splitlines()

In [32]:
words

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn',
 'abigail',
 'emily',
 'elizabeth',
 'mila',
 'ella',
 'avery',
 'sofia',
 'camila',
 'aria',
 'scarlett',
 'victoria',
 'madison',
 'luna',
 'grace',
 'chloe',
 'penelope',
 'layla',
 'riley',
 'zoey',
 'nora',
 'lily',
 'eleanor',
 'hannah',
 'lillian',
 'addison',
 'aubrey',
 'ellie',
 'stella',
 'natalie',
 'zoe',
 'leah',
 'hazel',
 'violet',
 'aurora',
 'savannah',
 'audrey',
 'brooklyn',
 'bella',
 'claire',
 'skylar',
 'lucy',
 'paisley',
 'everly',
 'anna',
 'caroline',
 'nova',
 'genesis',
 'emilia',
 'kennedy',
 'samantha',
 'maya',
 'willow',
 'kinsley',
 'naomi',
 'aaliyah',
 'elena',
 'sarah',
 'ariana',
 'allison',
 'gabriella',
 'alice',
 'madelyn',
 'cora',
 'ruby',
 'eva',
 'serenity',
 'autumn',
 'adeline',
 'hailey',
 'gianna',
 'valentina',
 'isla',
 'eliana',
 'quinn',
 'nevaeh',
 'ivy',
 'sadie',
 'piper',
 'lydia',
 'alexa',
 'josephine',
 'emery',
 'julia'

In [33]:
# vocabulary of characters and mappings to/form integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [34]:
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = t.tensor(X)
Y = t.tensor(Y)

In [35]:
X.shape,Y.shape

(torch.Size([32, 3]), torch.Size([32]))

In [36]:
C = t.randn((27,2), dtype=float)

In [37]:
C[5]

tensor([-0.6026,  0.8767], dtype=torch.float64)

In [38]:
C[t.tensor([5,6,7,7,4,6])]

tensor([[-0.6026,  0.8767],
        [-0.2720,  0.3582],
        [-0.7473,  0.2609],
        [-0.7473,  0.2609],
        [ 0.4457,  2.6749],
        [-0.2720,  0.3582]], dtype=torch.float64)

In [39]:
emb = C[X]

In [40]:
g = t.Generator().manual_seed(2147483647) # for reproducibility
C = t.randn((27, 2), generator=g)
W1 = t.randn((6, 100), generator=g)
b1 = t.randn(100, generator=g)
W2 = t.randn((100, 27), generator=g)
b2 = t.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [41]:
for p in parameters:
    p.requires_grad=True

In [42]:
for _ in range(10):
    emb = C[X]
    h = t.tanh(emb.view(-1,6)@W1+b1)
    logits = h@W2 + b2 
    counts = logits.exp()
    prob = counts/counts
    loss = f.cross_entropy(logits, Y)
    for p in parameters:
        p.grad = None
    loss.backward()
    # backward pass
    for p in parameters:
        p.data += -0.0001*p.grad
    
print(loss.item())

17.704740524291992


In [43]:
logits.max(1)

torch.return_types.max(
values=tensor([21.5207, 26.1639, 25.9041, 20.5114, 19.6184, 21.5207, 27.7807, 18.4060,
        17.8769, 12.2968, 16.3085, 14.7681, 21.5207, 20.6597, 14.9591, 17.8816,
        21.5207, 21.2410, 18.5668, 17.1871, 11.1012, 17.4663, 15.9138, 13.6529,
        15.6203, 21.5207, 20.9511, 14.3748, 14.6929, 23.1223, 15.9544, 24.1103],
       grad_fn=<MaxBackward0>),
indices=tensor([10, 10,  9, 14, 10, 10, 10, 26,  9, 21,  2, 14, 10, 10, 10, 22, 10, 10,
        10, 10, 14, 10, 26, 23, 20, 10, 10, 10, 23,  9, 14, 10]))